In [16]:
from math import log, sqrt, exp
import time
import datetime as dt                   # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr
from scipy.stats import norm  # normal cdf

from ib_insync import *
util.startLoop()

In [ ]:
# pull data from 
indicators = ['SPY']
end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=365)
raw_data = pdr.DataReader(indicators, 'yahoo', start_date, end_date)
data = raw_data['Adj Close']

In [3]:
std_SPY = data.tail(252).SPY.std()
mean_SPY = data.tail(252).SPY.mean()
print(f"Annual Volatility SPY: {std_SPY}")

Annual Volatility SPY: 30.322686983279574


In [3]:
try:
    ib.disconnect()
    time.sleep(5)
except:
    print("no ib connection to disconnect")

# initialize connection to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)  # IB Trader Workstation
#ib.connect('127.0.0.1', 4002, clientId=1)    # IB Gateway

<IB connected to 127.0.0.1:7497 clientId=12>

In [4]:
ib.positions()

[Position(account='DU6067498', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=80.0, avgCost=375.664125),
 Position(account='DU6067498', contract=Future(conId=495512551, symbol='ES', lastTradeDateOrContractMonth='20221216', multiplier='50', currency='USD', localSymbol='ESZ2', tradingClass='ES'), position=-5.0, avgCost=181472.85),
 Position(account='DU6067498', contract=Future(conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', currency='USD', localSymbol='VXV2', tradingClass='VX'), position=10.0, avgCost=29452.38),
 Position(account='DU6067498', contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), position=-10.0, avgCost=1114.77311)]

In [5]:
# create a contract for the underlying s&p500 index
spx = Stock('SPY', 'SMART', 'USD')
ib.qualifyContracts(spx)
ib.reqMarketDataType(4)

# grab the ticker
[ticker] = ib.reqTickers(spx)
spx_price = ticker.marketPrice()

In [6]:
# generate the option chain
opt_chain = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
chain = next(c for c in opt_chain if c.tradingClass == 'SPY' and c.exchange == 'SMART')

# filter strikes for those close to in the money
strikes = [s for s in chain.strikes
           if spx_price - 20 < s < spx_price + 20]

expirations = sorted(exp for exp in chain.expirations)[:3]
expirations = ['20221021']

rights = ['C']

contracts = [Option('SPY', expiration, strike, right, 'SMART', tradingClass='SPY')
        for right in rights
        for expiration in expirations
        for strike in strikes]
contracts = ib.qualifyContracts(*contracts)
ib.reqMarketDataType(4)

In [7]:
# grab an example contract
call = contracts[0]
[ticker] = ib.reqTickers(call)
ticker

Ticker(contract=Option(conId=568552091, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=342.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221021C00342000', tradingClass='SPY'), time=datetime.datetime(2022, 10, 10, 17, 59, 19, 66879, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=20.33, bidSize=113.0, ask=20.55, askSize=117.0, last=-1.0, lastSize=0.0, volume=0.0, close=23.33)

In [8]:
# now calculate the theoretical prices of these options using BS

def bs_price(S, K, T, r, sigma, opt="c"):
    """Price an option with the black-scholes model

    Args:
        S (float): Price of the underlying security
        K (float): Strike price
        T (float): time to maturity
        r (float): risk free interest rate
        sigma (float): volatility of the underlying security
        opt (str, optional): option type. Defaults to "C" (call)

    Returns:
        float: the black-scholes price of the option
    """
    d1 = (np.log(S / K) + (r + sigma**2 / 2.0) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if opt == "c":
        bs_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif opt == "p":
        bs_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        bs_price = None
    return bs_price


In [ ]:
price = spx_price
strike = ticker.contract.strike
maturity = 22/365
rfr = 0.036  # use shortterm libor!
vol = std_SPY / mean_SPY

opt_price = bs_price(price, strike, maturity, rfr, vol)

print(f"The close price of the option is: {ticker.close}")

3. At the beginning of the first trading day, sell 10 contracts of the call option with the largest mispricing. What would the proceeds from your trade be if the market price equals the BS option value? What are the actual proceeds from your trade?

In [10]:
ib.positions()

[Position(account='DU6067498', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=80.0, avgCost=375.664125),
 Position(account='DU6067498', contract=Future(conId=495512551, symbol='ES', lastTradeDateOrContractMonth='20221216', multiplier='50', currency='USD', localSymbol='ESZ2', tradingClass='ES'), position=-5.0, avgCost=181472.85),
 Position(account='DU6067498', contract=Future(conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', currency='USD', localSymbol='VXV2', tradingClass='VX'), position=10.0, avgCost=29452.38),
 Position(account='DU6067498', contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), position=-10.0, avgCost=1114.77311)]

The option delta is the sensitivity of the option to a change in price in the underlying security. 

$$
delta = \frac{\partial V}{\partial S}
$$


In [11]:
# get current positions
positions = [p for p in ib.positions() if p.contract.symbol == "SPY"]

# ensure that you have an open option position
opts = [p for p in positions if p.contract.secType == "OPT"]
if not opts:
    print("No option position to hedge")
option = opts[0]
con = option.contract

contracts = ib.qualifyContracts(con)
ib.reqMarketDataType(4)
[ticker] = ib.reqTickers(*contracts)
ticker

Ticker(contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), time=datetime.datetime(2022, 10, 10, 17, 59, 44, 267990, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=0.35, bidSize=13719.0, ask=0.36, askSize=8105.0, last=-1.0, lastSize=0.0, volume=0.0, close=0.56)

In [ ]:
# get the option delta from IB
delta = ticker.modelGreeks.delta
delta_neutral_pos = round(-option.position * delta * 100)

# calculate the difference between the hedge requirement and the current position
stks = [p for p in positions if p.contract.secType == "STK"]
if stks:
    hedge = delta_neutral_pos - stks[0].position
else:
    hedge = delta_neutral_pos
print(f"The deltahedge trade needed is SPY: {hedge}")

In [27]:
# execute the trade if the hedge outweighs the risk
if abs(hedge) > 10:
    if hedge > 10:
        action = "BUY"
    else:
        action = "SELL"
        
    # return contract and order required to remain delta neutral
    contract = Stock("SPY", "SMART", "USD")
    ib.qualifyContracts(contract)
    order = MarketOrder(action, abs(hedge))
    
    # place trade
    trade = ib.placeOrder(contract, order)
    assert order in ib.orders()
    assert trade in ib.trades()
    while not trade.isDone():
        ib.waitOnUpdate()
else:
    print("Already delta-neutral, no trade required.")

In [47]:
ib.positions()

[Position(account='DU6067707', contract=Option(conId=564970818, symbol='SPY', lastTradeDateOrContractMonth='20221021', strike=390.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   221021C00390000', tradingClass='SPY'), position=-10.0, avgCost=807.28017),
 Position(account='DU6067707', contract=Future(conId=540342682, symbol='VIX', lastTradeDateOrContractMonth='20221019', multiplier='1000', currency='USD', localSymbol='VXV2', tradingClass='VX'), position=10.0, avgCost=29502.38),
 Position(account='DU6067707', contract=Future(conId=495512551, symbol='ES', lastTradeDateOrContractMonth='20221216', multiplier='50', currency='USD', localSymbol='ESZ2', tradingClass='ES'), position=-5.0, avgCost=184072.85),
 Position(account='DU6067707', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=114.0, avgCost=370.3552756)]

Peer closed connection.


In [ ]:
ib.disconnect()